In [19]:
import tensorflow as tf
import pandas as pd
import math
from random import shuffle
import numpy as np

pd.set_option('display.max_columns', None)

In [20]:
train_data_load = pd.read_pickle("data/earlyAprilDataEnriched")

In [21]:
test_data_load = pd.read_pickle("data/lateAprilDataEnriched")

In [22]:
train_data_load[0:10]

,VendorID,lpep_pickup_datetime,Lpep_dropoff_datetime,Store_and_fwd_flag,RateCodeID,Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,Passenger_count,Trip_type,VincentyMiles,AvgTemp,Precipitation,NewSnow,trip_length_seconds,trip_length_minutes_rounded,pickup_minute,pickup_hour,pickup_day,pickup_weekday,dropoff_minute,dropoff_hour,dropoff_day,dropoff_weekday,pickup_sunrise,pickup_sunset,dropoff_sunrise,dropoff_sunset
4532400,2,2016-04-01 00:02:03,2016-04-01 00:05:53,0,1,-73.991180,40.685608,-73.984116,40.695980,1,1.0,0.806173,59.0,0.02,0.0,230.0,4.0,2,0,1,4,5,0,1,4,0.0,0.0,0.0,0.0
4532401,2,2016-04-01 00:01:31,2016-04-01 00:05:55,0,1,-73.844292,40.721432,-73.850441,40.724144,1,1.0,0.373149,59.0,0.02,0.0,264.0,4.0,1,0,1,4,5,0,1,4,0.0,0.0,0.0,0.0
4532402,2,2016-04-01 00:00:57,2016-04-01 00:07:36,0,1,-73.944008,40.714539,-73.938705,40.724926,1,1.0,0.768917,59.0,0.02,0.0,399.0,7.0,0,0,1,4,7,0,1,4,0.0,0.0,0.0,0.0
4532403,2,2016-04-01 00:01:22,2016-04-01 00:06:12,0,1,-73.952789,40.810749,-73.963509,40.796486,1,1.0,1.133387,59.0,0.02,0.0,290.0,5.0,1,0,1,4,6,0,1,4,0.0,0.0,0.0,0.0
4532404,2,2016-04-01 00:00:56,2016-04-01 00:05:25,0,1,-73.991249,40.691433,-73.988762,40.683598,3,1.0,0.556217,59.0,0.02,0.0,269.0,4.0,0,0,1,4,5,0,1,4,0.0,0.0,0.0,0.0
4532405,2,2016-04-01 00:00:47,2016-04-01 00:14:49,0,1,-73.968704,40.677856,-73.935036,40.651569,1,1.0,2.533694,59.0,0.02,0.0,842.0,14.0,0,0,1,4,14,0,1,4,0.0,0.0,0.0,0.0
4532406,2,2016-04-01 00:00:07,2016-04-01 00:03:41,0,1,-73.957878,40.711040,-73.955887,40.707653,1,1.0,0.256062,59.0,0.02,0.0,214.0,4.0,0,0,1,4,3,0,1,4,0.0,0.0,0.0,0.0
4532407,2,2016-04-01 00:00:13,2016-04-01 00:18:43,0,1,-73.960648,40.719345,-73.917854,40.781212,1,1.0,4.823582,59.0,0.02,0.0,1110.0,18.0,0,0,1,4,18,0,1,4,0.0,0.0,0.0,0.0
4532408,2,2016-04-01 00:00:34,2016-04-01 00:12:19,0,1,-73.984062,40.676144,-74.027718,40.632233,1,1.0,3.800473,59.0,0.02,0.0,705.0,12.0,0,0,1,4,12,0,1,4,0.0,0.0,0.0,0.0
4532409,2,2016-04-01 00:47:13,2016-04-01 00:59:30,0,1,-73.884544,40.755604,-73.856026,40.745335,1,1.0,1.655823,59.0,0.02,0.0,737.0,12.0,47,0,1,4,59,0,1,4,0.0,0.0,0.0,0.0


In [23]:
def cleanData(orig_data, get_minutes = False):
    label_column = "trip_length_seconds"
    if get_minutes:
        labels = "trip_length_minutes_rounded"
    labels_array = orig_data[label_column].copy().values
    labels = np.reshape(labels_array, (len(labels_array), 1))
    clean_data = orig_data.drop(labels = ["lpep_pickup_datetime", "Lpep_dropoff_datetime", "trip_length_seconds", "trip_length_minutes_rounded"], axis = 1)
    return (clean_data.values, labels)

In [24]:
try_with_minutes = False

In [25]:
train_data, train_labels = cleanData(train_data_load, try_with_minutes)

In [26]:
test_data, test_labels = cleanData(test_data_load, try_with_minutes)

In [54]:
def buildNeuralNetAndGetAccuracy(data_for_training, a_train, data_for_testing, a_test):
    #Only two layers
    batch_size = 300
    iterations = 501
    
    inputFeatureSize = len(data_for_training[0])
    print("There are {} features".format(inputFeatureSize))
    hidden_units1 = 25
    hidden_units2 = 25
    output_size = 1
            
    input_placeholder = tf.placeholder(tf.float32, shape=(None, inputFeatureSize), name = "input_placeholder")
    labels_placeholder = tf.placeholder(tf.float32, shape=(None, 1), name = "labels_placeholder")
    
    #remember that we need W of small random values for gradient descent
    hiddenLayer1W = tf.Variable(tf.truncated_normal([inputFeatureSize, hidden_units1], stddev=1.0 / math.sqrt(float(inputFeatureSize))))
    hiddenLayer2W = tf.Variable(tf.truncated_normal(shape=(hidden_units1, hidden_units2), stddev=1.0 / math.sqrt(float(inputFeatureSize))))
    outW = tf.Variable(tf.truncated_normal(shape=(hidden_units2, output_size), stddev=1.0 / math.sqrt(float(inputFeatureSize))))

    hiddenLayer1b = tf.Variable(tf.truncated_normal([hidden_units1], stddev=1.0 / math.sqrt(float(inputFeatureSize))))
    hiddenLayer2b = tf.Variable(tf.truncated_normal([hidden_units2], stddev=1.0 / math.sqrt(float(inputFeatureSize))))
    outb = tf.Variable(tf.truncated_normal([output_size], stddev=1.0 / math.sqrt(float(inputFeatureSize))))

    hiddenLayer1 = tf.tanh(tf.matmul(input_placeholder, hiddenLayer1W) + hiddenLayer1b)
    hiddenLayer2 = tf.tanh(tf.matmul(hiddenLayer1, hiddenLayer2W) + hiddenLayer2b)
    p_out = tf.matmul(hiddenLayer2, outW) + outb
        
#     loss = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(tf.log(tf.add(p_out, tf.ones([len(p_out), 1]))), tf.log(tf.add(a_train, tf.ones([len(a_train), 1])))))))
    loss = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(tf.log(tf.add(p_out, tf.ones(tf.stack([tf.shape(p_out)[0], 1])))), tf.log(tf.add(labels_placeholder, tf.ones(tf.stack([tf.shape(labels_placeholder)[0], 1]))))))))

    # Other optimizers - https://www.tensorflow.org/api_guides/python/train#Optimizers
    optimizer = tf.train.GradientDescentOptimizer(0.25)
    
    global_step_counter = tf.Variable(0, trainable=False)
    train_op = optimizer.minimize(loss, global_step=global_step_counter)
    
    saver = tf.train.Saver()
    
    with tf.Session() as sess:
        init = tf.global_variables_initializer()
        sess.run(init)
        
        for i in range(0,iterations):
            indicies = list(range(0,len(data_for_training)))
            shuffle(indicies)
            indicies100 = indicies[0:batch_size]
            batch_xs = [data_for_training[j] for j in indicies100]
            batch_ys = [a_train[j] for j in indicies100]
            _, loss_value = sess.run([train_op, loss],
                             feed_dict={input_placeholder: batch_xs, labels_placeholder: batch_ys})
            if(i%500 == 0):
                print("Alive and well! {} Loss: {:.2f}".format(i, loss_value))
                saver.save(sess, 'C:\\Users\\Brittany\\Dropbox\\School\\CMSC678\\machine-learning\\saved_models\\first_model',global_step=i)
                
        print("TRAINED!")
        
        # Calculate accuracy
        print(p_out.shape())
        accuracy = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(tf.log(tf.add(p_out, tf.ones(tf.stack([tf.shape(p_out)[0], 1])))), tf.log(tf.add(labels_placeholder, tf.ones(tf.stack([tf.shape(labels_placeholder)[0], 1]))))))))
        return accuracy.eval({input_placeholder: data_for_testing, labels_placeholder: a_test})

In [53]:
accResults = buildNeuralNetAndGetAccuracy(train_data, train_labels, test_data, test_labels)

Alive and well! 0 Loss: 6.09


KeyboardInterrupt: 